#  Dimensionality Reduction and Visualization

Example from sklearn documentation.
Dimensionality reduction is the task of deriving a set of new
artificial features that is smaller than the original feature
set while retaining most of the variance of the original data.
Here we'll use a common but powerful dimensionality reduction
technique called Principal Component Analysis (PCA).
We perform dimensionality reduction using first the singular value decomposition of the centered matrix and after the one computed by the PCA subroutine of sklearn.
We work on the iris dataset:

In [168]:
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target
%pylab inline
# we plot the dataset composed on 3 column using a three dimensional space
US = np.copy(X)
X.shape


Populating the interactive namespace from numpy and matplotlib


/Users/frances/opt/anaconda3/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['f', 'e']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


(150, 4)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

target=iris.target
#ax = fig.gca(projection='3d')
fig = figure()
ax = fig.add_subplot(projection='3d')

ax.plot(US[target==0,0],US[target==0,1],US[target==0,2],'bo')
ax.plot(US[target==1,0],US[target==1,1],US[target==1,2],'ro')
ax.plot(US[target==2,0],US[target==2,1],US[target==2,2],'go')

PCA is performed using linear combinations of the original features
using a  Singular Value Decomposition of the matrix X so
as to project the data onto a base of the top singular vectors.
If the number of retained components is 2 or 3, PCA can be used
to visualize the dataset:

In [ ]:
from sklearn.decomposition import PCA
whiten=True
ncomp=2
pca = PCA(copy=True,n_components=ncomp,whiten=whiten).fit(X)
# copy = True the matrix  X is not modified
# n_components = number of computed singular values 
# withen = TRUE  the components are scaled to have unit variance 

Once fitted, the pca model exposes the singular vectors in the components_ attribute:

In [ ]:
# direction of maximal variance are the vectors of the matrix V of the SVD
# this are the rows of V^T
pca.components_                           

Let us project the iris dataset along those first two dimensions:

In [ ]:
X_pca = pca.transform(X)
X_pca.shape

Let us compare this results with the singular value decomposition results

In [ ]:
import scipy.linalg as la
AS = np.copy(np.asarray(X))
(nb, na) = AS.shape

meanAS=np.mean(AS,axis=0).reshape(1,na)

print('mean of the columns',meanAS)

In [ ]:
# subtract the mean 
e = np.ones((nb,1))  
AS = AS - np.dot(e,meanAS) # outer product using matrix operation
print('mean of the columns after ', np.mean(AS,axis=0)) 
As1 = np.copy(np.asarray(X))
As1 -=  meanAS # compacted outer product it is the same as using the outer product
print('mean of the columns after ', np.mean(As1,axis=0)) 

In [ ]:
# compute the SVD
U, s, Vh = la.svd(AS,full_matrices=False)

US=np.copy(U)
if whiten:
 US = US[:,0:ncomp]*s[0:ncomp]
 stdUS = np.std(US,axis=0,ddof=1) # the standard deviation is computed by dividing by sqrt(N-ddof)
 US = US/stdUS
else:
 US = US[:,0:ncomp]*s[0:ncomp]

print('Vh')
print(Vh[0:2,:])

print('pca_components')
print(pca.components_)
# to check the equality we compare the ratio element by element
print('X_pca ratio  with  U_i*s_i')
print(np.max(abs(X_pca/(US[:,0:ncomp]))),np.min(abs(X_pca/(US[:,0:ncomp]))))


Using withen=True we have performed a standarditazion of the dataset, which means that the data are now centered  with unit variance:

In [ ]:
import numpy as np
X_pca.mean(axis=0)

In [ ]:
np.round(X_pca.std(axis=0,ddof=1), decimals=15)

Furthermore the samples components do no longer carry any linear correlation, the correlation matrix is the identity matrix:

In [ ]:
np.round(np.corrcoef(X_pca.T), decimals=5)

In thi example the covariance matrix is equal to the correlation matrix

In [ ]:
np.round(np.cov(X_pca.T), decimals=5)

The means of the columns is 0

In [ ]:
np.mean(X_pca,axis=0)

In [ ]:
np.round((X_pca.T.dot(X_pca))/(nb-1), decimals=5)

Now we can visualize the results using the following utility function:

In [ ]:
from itertools import cycle

def plot_PCA_1D(data, target, target_names):
    colors = cycle('rgbcmykw')
    target_ids = range(len(target_names))
    figure()
    for i, c, label in zip(target_ids, colors, target_names):
          plot(data[target == i, 0],
                   c=c, label=label,marker='o',linestyle = 'None')
    legend()

def plot_PCA_2D(data, target, target_names):
    colors = cycle('rgbcmykw')
    target_ids = range(len(target_names))
    figure()
    for i, c, label in zip(target_ids, colors, target_names):
          scatter(data[target == i, 0], data[target == i, 1],
                   c=c, label=label)
    legend()

Now calling this function for our data, we see the plot:

In [ ]:
plot(X_pca[:,0],marker='o',linestyle = 'None')

In [ ]:
plot_PCA_1D(X_pca, iris.target, iris.target_names)

Note that this projection was determined *without* any information about the
labels (represented by the colors): this is the sense in which the learning
is unsupervised.  Nevertheless, we see that the projection gives us insight
into the distribution of the different flowers in parameter space: notably,
*iris setosa* is much more distinct than the other two species.

In [ ]:
plot_PCA_2D(X_pca, iris.target, iris.target_names)

Note also that the default implementation of PCA computes the SVD of the full
data matrix, which is not scalable when both ``n_samples`` and
``n_features`` are big (more that a few thousands).
If you are interested in a number of components that is much
smaller than both ``n_samples`` and ``n_features``, consider using
 special solvers for svd 'arpack' or 'randomized'.

In [ ]:
pca = PCA(copy=True,n_components=3, whiten=True,svd_solver='randomized').fit(X)
US = pca.transform(X)
from mpl_toolkits.mplot3d import Axes3D
target=iris.target
fig = figure()
ax = fig.add_subplot(projection='3d')
ax.plot(US[target==0,0],US[target==0,1],US[target==0,2],'ro')
ax.plot(US[target==1,0],US[target==1,1],US[target==1,2],'go')
ax.plot(US[target==2,0],US[target==2,1],US[target==2,2],'bo')


In [ ]:
plot_PCA_1D(X_pca, iris.target, iris.target_names)

In [ ]:
plot_PCA_2D(X_pca, iris.target, iris.target_names)

### Exercise:

Repeat the above dimensionality reduction with
``Randomized`` or "arpack".

You can re-use the ``plot_PCA_2D`` function from above.
Are the results similar to those from standard PCA?

In [ ]:
import time
ns = 2
t1=time.process_time()
pca = PCA(copy=True,n_components=ns).fit(X)
US = pca.transform(X)
t2d=time.process_time()-t1
print( "clock  default = ", t2d )

t1=time.process_time()
pca = PCA(copy=True,n_components=ns,svd_solver='randomized').fit(X)
US = pca.transform(X)
t2d=time.process_time()-t1
print( "clock  randomized = ", t2d )


t1=time.process_time()
pca = PCA(copy=True,n_components=ns,svd_solver='arpack').fit(X)
US = pca.transform(X)
t2d=time.process_time()-t1
print( "clock  arpack= ", t2d )

#apply arpack PCA to the iris data as above, and plot the result.


Check the projection on columns

In [ ]:
Asat=np.dot(U[:,0:4].T,AS)

the observed values are reconstructed by adding the mean

In [ ]:
Xappr = np.dot(U[:,0:4],Asat) + np.dot(e,meanAS)

In [ ]:
np.linalg.norm(Xappr-X,2)

In [ ]:
simg = s
f=simg**2. / np.sum(simg**2.)
r = simg.shape
entropy = (-1/np.log(r))*np.sum(f*np.log(f))
ks = int(r*entropy)
  #print('Entropy =',entropy, 'suggested k=', ks , simg[0:ks],)
perc=(1-1e-4)
nc = int(r*entropy*perc)+1   
print('Entropy =',entropy, 'suggested k=', nc,r*entropy*perc,r,np.log(r))

In [ ]:
s
